In [1]:
import collections
import itertools
import heapq

In [2]:
testlines = '''###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############'''.splitlines()

In [3]:
with open('day20input.txt') as fp:
    data = fp.read().splitlines()

## Part 1 ##

Yet another pathfinding problem, now needing breadcrumbs (I think).
https://aoc.just2good.co.uk/python/shortest_paths#breadcrumb-trail

In [4]:
def parse_input(lines):
    walls = set()
    size = (len(lines), len(lines[0]))
    for row, line in enumerate(lines):
        for col, c in enumerate(line):
            match c:
                case '#':
                    walls.add((row, col))
                case '.':
                    pass
                case 'S':
                    start = (row, col)
                case 'E':
                    end = (row, col)
                case _:
                    raise ValueError('Invalid character in input: ', c)
    return start, end, walls, size

In [5]:
start, end, walls, size = parse_input(testlines)

In [6]:
DIRS = [(0,+1), (+1, 0), (0, -1), (-1, 0)] # E, S, W, N
def get_shortest_path(start, end, walls, size):
    frontier = collections.deque()
    frontier.append(start)
    came_from = {}
    came_from[start] = None
    while frontier:
        current = frontier.popleft()
        if current == end:
            break
        for d in DIRS:
            newpos = (current[0] + d[0], current[1] + d[1])
            if newpos in walls:
                continue
            if newpos not in came_from:
                if (0 <= newpos[0] < size[0]) and (0 <= newpos[1] < size[1]):
                    frontier.append(newpos)
                    came_from[newpos] = current
    if current != end:
        raise ValueError('Path not found from start to end')
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path

In [7]:
path = get_shortest_path(start, end, walls, size)
len(path) - 1

84

In [8]:
def traverse(start, end, walls, size):
    q = [(0, start)]
    seen = set()
    while q:
        cost, pos = heapq.heappop(q)
        if pos in seen:
            continue
        if end == pos:
            return cost
        seen.add(pos)
        # add possible next moves to the queue
        for d in DIRS:
            newpos = (pos[0]+d[0], pos[1]+d[1])
            if newpos in walls:
                continue
            if (0 <= newpos[0] < size[0]) and (0 <= newpos[1] < size[1]):
                heapq.heappush(q, (cost+1, newpos))
    raise ValueError('Heap queue exhausted without finding the target')

In [9]:
traverse(start, end, walls, size)

84

In [10]:
def count_cheats(path, walls, size, threshold=0):
    t_no_cheat = len(path) - 1
    count = 0
    seen = set()
    for pos in path:
        for d in DIRS:
            p1 = (pos[0] + d[0], pos[1] + d[1])
            p2 = (pos[0] + 2*d[0], pos[1] + 2*d[1])
            if (p1, p2) in seen:
                continue
            if (p2[0] < 0) or (p2[0] >= size[0]) or (p2[1] < 0) or (p2[1] >= size[1]):
                continue
            if (p1 in walls) and (p2 in path) and (path.index(p2) > path.index(pos)):
                # potential cheat found
                seen.add((p1, p2))
                cheat_walls = walls.copy()
                cheat_walls.remove(p1)
                t_cheat = traverse(path[0], path[-1], cheat_walls, size)
                if t_no_cheat - t_cheat >= threshold:
                    count += 1
    return count

In [11]:
count_cheats(path, walls, size)

44

In [12]:
def part1(lines, threshold):
    start, end, walls, size = parse_input(lines)
    path = get_shortest_path(start, end, walls, size)
    num_cheats = count_cheats(path, walls, size, threshold)
    return num_cheats

In [13]:
part1(testlines, 0)

44

In [14]:
part1(data, 100)

1367

My count_cheats function works, but it's very slow, taking several minutes to run. I knew p2 had to be on the path,
and further along than pos, but didn't think that it just meant I could count all shortcuts where pos and p2 have a
taxicab distance of 2. Oops. So, now stealing from https://www.reddit.com/r/adventofcode/comments/1hicdtb/comment/m2y56t8/

In [17]:
def part1and2(lines):
    start, end, walls, size = parse_input(lines)
    path = get_shortest_path(start, end, walls, size)
    count1 = count2 = 0
    for i,j in itertools.combinations(range(len(path)), 2):
        pi, pj = path[i], path[j]
        dij = abs(pi[0] - pj[0]) + abs(pi[1] - pj[1])
        pathdist = j - i
        if 2 == dij and pathdist - dij >= 100:
            count1+= 1
        if dij <= 20 and pathdist - dij >= 100:
            count2+= 1
    return count1, count2

In [18]:
part1and2(data)

(1367, 1006850)